# Exogenizing an endogenous equation  

World Bank models are comprised of estimated equations, identities and exogenous variables. 


As constructed behavioural equations can be activated or de-activated -- turned on or turned off.  When a behavioral equation is turned off, its dependent variable is described as having been exogenized.  Its value is no longer determined by the model, but can be set to arbitrary values just like any exogenous variable. 

By definition identities must always be true and therefore should never be exogenized.

This short note describes the process to follow to exogenize a variable using `modelflow`.

# The simulation

This note will run two simulations.  One shocking the value of the policy rate for three years, and the second running the same shock but freezing some other variables in order to explore their role in the outcome.



## Set up python session

As a necessary first step, the python sessions has to be initialized and the modelclass and otehr support libraries to be used imported.

In [97]:
 %load_ext autoreload
 %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import pandas as pd
from modelclass import model 
import modelwidget as mw
model.widescreen()
model.scroll_off()

## Load a pre-existing model, data and descriptions 


In [99]:
mpak,result = model.modelload('../models/pak.pcim',run=1,silent=1,keep='Baseline no fix')
mpak.basedf=result

file read:  C:\mflow\modelflow-manual\papers\mfbook\content\models\pak.pcim


First we use the variable interrogation functions of `modelflow` to determine the mnemonic of the variable that we want to shock.


In [100]:
mpak['!*policy*'].des

PAKFMLBLPOLYXN        : Key Policy Interest Rate
PAKFMLBLPOLYXN_A      : Add factor:Key Policy Interest Rate
PAKFMLBLPOLYXN_D      : Fix dummy:Key Policy Interest Rate
PAKFMLBLPOLYXN_FITTED : Fitted  value:Key Policy Interest Rate
PAKFMLBLPOLYXN_X      : Fix value:Key Policy Interest Rate
PAKINTRDDIFF          : Domestic Interest Rate Spread Over Policy Rate
PAKINTRDDIFF_A        : Add factor:Domestic Interest Rate Spread Over Policy Rate
PAKINTRDDIFF_D        : Fix dummy:Domestic Interest Rate Spread Over Policy Rate
PAKINTRDDIFF_FITTED   : Fitted  value:Domestic Interest Rate Spread Over Policy Rate
PAKINTRDDIFF_X        : Fix value:Domestic Interest Rate Spread Over Policy Rate
PAKINTREDIFF          : External Interest Rate Spread Over Policy Rate
PAKINTREDIFF_A        : Add factor:External Interest Rate Spread Over Policy Rate
PAKINTREDIFF_D        : Fix dummy:External Interest Rate Spread Over Policy Rate
PAKINTREDIFF_FITTED   : Fitted  value:External Interest Rate Spread Over Pol

To determine if `PAKFMLBLPOLYXN` is a endogenous or exogenous variable we can interggoate the mpak.endogene list by engquiring if the variable `PAKFMLBLPOLYXN` in the list of endogenous variables `mpak.endogene`.

In [101]:
'PAKFMLBLPOLYXN' in mpak.endogene


True

It is (the query returned True), so we can look at its equation using the `.frml` or `.show` functions.


In [102]:
mpak['PAKFMLBLPOLYXN'].frml

PAKFMLBLPOLYXN : FRML <DAMP,STOC> PAKFMLBLPOLYXN = (100*PAKFMLBLPOLYXN_A+100* (0.900155952396245*PAKFMLBLPOLYXN(-1)/100+(1-0.900155952396245)*(PAKMPPOLNATRXN/100+1.2*(((LOG(PAKNECONPRVTXN))-(LOG(PAKNECONPRVTXN(-1))))-PAKINFLEXPT/100)+0.5*PAKNYGDPGAP_/100)) ) * (1-PAKFMLBLPOLYXN_D)+ PAKFMLBLPOLYXN_X*PAKFMLBLPOLYXN_D $


# Preparing the initial simulation

Given that `PAKFMLBLPOLYXN` is an endogenous equation we can shock it any number of ways. Lets exogenize it for the three year period 2025-2027 setting it a level of 100 basis points higher than normal.

Before proceeding we should determine its units. i.e. is a 7 percent interest expressed as 7 or .07.



In [103]:
with mpak.set_smpl(2020,2030):
    print(mpak['PAKFMLBLPOLYXN'].df)

      PAKFMLBLPOLYXN
2020        6.670351
2021        7.056445
2022        7.287498
2023        7.376232
2024        7.362666
2025        7.285334
2026        7.171706
2027        7.039316
2028        6.899970
2029        6.763110
2030        6.637192


Its the former so we will want to add 1 to the value of the variable.

In modelflow every equation of a variable that can be exogenized is written in the form:

`<frml <z,exo> VAR  = (expression)*(1-VAR_D) + VAR_X * VAR_D`

In this formulation the variablre `VAR_D` effectively act as switch, when `VAR_D=1` the expression simplifies to `VAR=VAR_X`, if `VAR_D=0` the equation simplifies to `(expression)`, which in World Bank models is the estimated equation brought in from EViews.


# Step by Step exogenization of an endogenous variable

Having determiend that `PAKFMLBLPOLYXN` is a behavioural variable and that we want to exogenize it we must:

1. Reset the `PAKFMLBLPOLYXN_D` to 1
2. Set the `PAKFMLBLPOLYXN_X` variable to the same value as it is in the current model
3. Solve the model with these settings
4. Make sure it returns the same results as before.

The modelflow method `.fix()` will automatically do steps 1 and 2 (they can also be done manually).

Below the command `fix_df = mpak.fix(result,var_to_be_fixed,2020,2100)` takes the DataFrame from our inital load `result` as an input, and then changes the `_X` and `_D` versions of the variables listed in the string var_to_be_fixed.  In this case the single var `PAKFMLBLPOLYXN` but as the commented line above indicates the same operation could be performed on multiple variables simultaneously if that was our desire. Finally the operation is performed for data in the years between 2020 and 2100 inclusive.

The result of the operation is a new `DataFrame` `fix_df` which is acopyu of result but with the above transformation performed.


In [104]:
#var_to_be_fixed = 'PAKGGEXPCAPTCN PAKGGEXPGNFSCN PAKGGEXPOTHRCN PAKGGEXPTRNSCN'

mpak.set_smpl(2000,2100)
var_to_be_fixed = 'PAKFMLBLPOLYXN'
fix_df = mpak.fix(result,var_to_be_fixed,2025,2027)

The folowing variables are fixed
PAKFMLBLPOLYXN


In [105]:
with mpak.set_smpl(2020,2030):
    print(mpak['PAKFMLBLPOLYXN*'].df)


      PAKFMLBLPOLYXN  PAKFMLBLPOLYXN_A  PAKFMLBLPOLYXN_D  \
2020        6.670351         -0.004564               0.0   
2021        7.056445         -0.003879               0.0   
2022        7.287498         -0.003297               0.0   
2023        7.376232         -0.002803               0.0   
2024        7.362666         -0.002382               0.0   
2025        7.285334         -0.002025               0.0   
2026        7.171706         -0.001721               0.0   
2027        7.039316         -0.001463               0.0   
2028        6.899970         -0.001244               0.0   
2029        6.763110         -0.001057               0.0   
2030        6.637192         -0.000898               0.0   

      PAKFMLBLPOLYXN_FITTED  PAKFMLBLPOLYXN_X  
2020               7.126715               0.0  
2021               7.444354               0.0  
2022               7.617221               0.0  
2023               7.656496               0.0  
2024               7.600891            

:::{note}

**.fix(dataframe,variable pattern, start,end)** Returns a dataframe where variables matching pattern are fixed.

The code fixes the values for some variables by setting the \_X variable to the current value, and \_D variable to 1 over the timespan \[start,end\].
:::

## Step three is to solve the model with this new dataframe

We then solve the model with the new dataframe which should return exactly the same results as before.

In [113]:
mpak.set_smpl(2000,2100)
res = mpak(fix_df,silent=1,keep=f'Baseline') 

with mpak.set_smpl(2020,2035):
    print(round(mpak['PAKNYGDPMKTPKN PAKFMLBLPOLYXN PAKNECONPRVTKN PAKNEGDIFPRVKN'].difpctlevel.df,7))


      PAKNYGDPMKTPKN  PAKFMLBLPOLYXN  PAKNECONPRVTKN  PAKNEGDIFPRVKN
2020             0.0             0.0             0.0             0.0
2021             0.0             0.0             0.0             0.0
2022             0.0             0.0             0.0             0.0
2023             0.0             0.0             0.0             0.0
2024             0.0             0.0             0.0             0.0
2025             0.0             0.0             0.0             0.0
2026             0.0             0.0             0.0             0.0
2027             0.0             0.0             0.0             0.0
2028             0.0             0.0             0.0             0.0
2029             0.0             0.0             0.0             0.0
2030             0.0             0.0             0.0             0.0
2031             0.0             0.0             0.0             0.0
2032             0.0             0.0             0.0             0.0
2033             0.0             0

# Prepare the simulation

The objective is to temporarily exogenize the monetary policy interest rate and increase it from its pre-shock level by 100 basis points.  This can be done with the `.upd()` method. Here the results are assigned to the DataFrame `MPShockdf`.

In [115]:
mpak.set_smpl(2000,2100)
MPShockdf  =  fix_df.upd('''                          
                           <2025 2028> PAKFMLBLPOLYXN_X + 1
                           ''')

Next we solve the model, by inputting the MPShockdf and storing the results in MPShockRes, while also using the keep option.

In [116]:
MPShockRes = mpak(MPShockdf,silent=1,keep=f'Monetary Policy hike _100 BP 2025-2027',alfa = 0.7)

### Examine the results

Below, the `difpctlevel.mul100.df` transformation displays the percentage change in the level of the displayed variables as compared with their pre-shock levels $\bigg(\frac{X^{shock}_t}{X^{base}_t}-1\bigg) * 100$.

In [118]:
with mpak.set_smpl(2020,2035):
#    display(mpak['PAKNYGDPMKTPKN PAKFMLBLPOLYXN PAKNECONPRVTKN PAKNEGDIFPRVKN PAKNEGDIFGOVKN PAKNECONPRVTXN'])
    print(round(mpak['PAKNYGDPMKTPKN PAKFMLBLPOLYXN PAKNECONPRVTKN PAKNEGDIFPRVKN PAKNEGDIFGOVKN PAKNECONPRVTXN'].difpctlevel.mul100.df,2))

      PAKNYGDPMKTPKN  PAKFMLBLPOLYXN  PAKNECONPRVTKN  PAKNEGDIFPRVKN  \
2020            0.00            0.00            0.00            0.00   
2021            0.00            0.00            0.00            0.00   
2022            0.00            0.00            0.00            0.00   
2023            0.00            0.00            0.00            0.00   
2024            0.00            0.00            0.00            0.00   
2025            0.07           13.73           -0.27           10.34   
2026           -0.46           13.94           -0.84           10.57   
2027           -0.76           14.21           -1.16           10.95   
2028           -0.90           11.98           -1.25            9.44   
2029           -0.83            9.87           -1.12            8.00   
2030           -0.65            8.03           -0.89            6.74   
2031            0.00            0.00            0.00            0.00   
2032            0.00            0.00            0.00            

### More information about a simulation

The `.fix_inf()` method will show details about exogenized or 'fixed' variables, including the levels of the \_A \_D \_X variants of the dependent variable.  Show relevant variables for each fixed variables
To make it short we only show the first 4 years  

In [49]:
with mpak.set_smpl(2024,2030):
    mpak.fix_inf()


Key Policy Interest Rate

FRML <DAMP,STOC> PAKFMLBLPOLYXN = (100*PAKFMLBLPOLYXN_A+100* (0.900155952396245*PAKFMLBLPOLYXN(-1)/100+(1-0.900155952396245)*(PAKMPPOLNATRXN/100+1.2*(((LOG(PAKNECONPRVTXN))-(LOG(PAKNECONPRVTXN(-1))))-PAKINFLEXPT/100)+0.5*PAKNYGDPGAP_/100)) ) * (1-PAKFMLBLPOLYXN_D)+ PAKFMLBLPOLYXN_X*PAKFMLBLPOLYXN_D  $

FRML <CALC> PAKFMLBLPOLYXN_A = PAKFMLBLPOLYXN/100- ((0.900155952396245*PAKFMLBLPOLYXN(-1)/100+(1-0.900155952396245)*(PAKMPPOLNATRXN/100+1.2*(((LOG(PAKNECONPRVTXN))-(LOG(PAKNECONPRVTXN(-1))))-PAKINFLEXPT/100)+0.5*PAKNYGDPGAP_/100))) $


,PAKFMLBLPOLYXN,PAKFMLBLPOLYXN_X,PAKFMLBLPOLYXN_D,PAKFMLBLPOLYXN_A
2024,7.362666,7.362666,1,-0.002382
2025,8.285334,8.285334,1,0.007903
2026,8.171706,8.171706,1,-0.000522
2027,8.039316,8.039316,1,0.000086
2028,6.899970,6.899970,1,-0.009445
2029,6.763110,6.763110,1,-0.000461
2030,6.637192,6.637192,1,-0.000654


### Additional methods related to exogenized variables.

endogenous The model object maintains several lists relevant to endogenous variables, listed in the below table. The first column indicates the role of the variables about which information is to be returned.  The second, the suffix that is attached to root name of the endogenous variable to indicate that role. The third column gives information about all variables in the model object that are fixable, while the final column are similar lists but indicates only those variables that are currently exogenized (fixed).


|Variable type|variable suffix|Lists of all "fixable" variables|List of *fixed* variables|
|:--|:--|:--|:--|    
Endogenous|  |.fix_endo|.fix_endo_fixed
Dummy|_D|.fix_dummy|.fix_dummy_fixed
Fixed value|_X|.fix_value|.fix_value_fixed
Add factor|_A|.fix_add_factor|.fix_add_factor_fixed

#### LIst of all fixable variables for this model 

Below a list of the first 10 endogenous variables in the mpak model object (the \[:10\] restricts the output to the first ten variables). 

In [119]:
mpak.fix_endo[:10] # The [:10] truncates the results to the last 10 endogenous variables in the model.

['PAKBMFSTOTHRCD',
 'PAKBMFSTREMTCD',
 'PAKBMGSRGNFSCD',
 'PAKBMGSRMRCHCD',
 'PAKBXFSTOTHRCD',
 'PAKBXFSTREMTCD',
 'PAKBXGSRGNFSCD',
 'PAKBXGSRMRCHCD',
 'PAKCO2EM',
 'PAKFMLBLMTWOKN']

#### List of fixed variables

Below a list of all the endogenous variables that are actually exogenized (fixed) in the 'mpak' model object at this point in time.

In [121]:
mpak.fix_endo_fixed

['PAKFMLBLPOLYXN']